In [1]:
import pyiqfeed as iq
import pandas as pd
import datetime
import redis

In [2]:
redis_host = "127.0.0.1" #'dt-tor-03422.cppib.ca' 
redis_port = 6379
redis_db = 0
redis_server = redis.StrictRedis(host=redis_host, port=redis_port, db=redis_db, ssl=False)

In [3]:
ASSET_NAME = "name"
DATE_NAME = "date"
TIME_NAME = "time"
DATETIME_NAME = "datetime"
OPEN_NAME = "open"
HIGH_NAME = "high"
LOW_NAME = "low"
CLOSE_NAME = "close"
VOLUME_NAME = "volume"
VOLUME_TOTAL_NAME = "volume_total"
OPEN_INTEREST_NAME = "open_interest"
TRADE_NUM_NAME = "trade_number"

DICT_NAME_MAPPER = {
    "date": DATE_NAME,
    "open_p": OPEN_NAME,
    "high_p": HIGH_NAME,
    "low_p": LOW_NAME,
    "close_p": CLOSE_NAME,
    "prd_vlm": VOLUME_NAME,
    "open_int": OPEN_INTEREST_NAME,
    "tot_vlm": VOLUME_TOTAL_NAME,
    "num_trds": TRADE_NUM_NAME,
    "time": TIME_NAME,
    "last_sz": VOLUME_NAME,
    "last": CLOSE_NAME,
}

In [4]:
def launch_service(dtn_login, dtn_password, dtn_product_id):
    """Check if IQFeed.exe is running and start if not"""

    svc = iq.FeedService(
        product=dtn_product_id,
        version="Debugging",
        login=dtn_login,
        password=dtn_password,
    )
    svc.launch(headless=False)
from localconfig.passwords import dtn_login, dtn_password, dtn_product_id

launch_service(dtn_login, dtn_password, dtn_product_id)

In [11]:
end_date = datetime.datetime.today()
start_date = datetime.datetime.today() - pd.offsets.BDay(50)

list_tickers = redis_server.keys()
# list_tickers = [
#     "AAPL",
#     "AMGN",
#     "AXP",
#     "BA",
#     "CAT",
#     "CRM",
#     "CSCO",
#     "CVX", 
#     "DIS",
#     "DOW",
#     "GS",
#     "HD",
#     "HON",
#     "EWZ",
#     "AUDCAD.FXCM",
#     "AUDCHF.FXCM",
#     "AUDJPY.FXCM",
#     "AUDNZD.FXCM",
#     "AUDUSD.FXCM",
#     "CADCHF.FXCM",
#     "CHFJPY.FXCM",
#     "EURAUD.FXCM",
#     "EURCAD.FXCM",
#     "EURCHF.FXCM",
#     "EURGBP.FXCM",
#     "EURJPY.FXCM",
#     "EURNZD.FXCM",
#     "EURUSD.FXCM",
#     "GBPAUD.FXCM",
#     "GBPCAD.FXCM",
#     "GBPCHF.FXCM",
#     "GBPJPY.FXCM",
#     "GBPNZD.FXCM",
#     "GBPUSD.FXCM",
#     "NZDCAD.FXCM",
#     "NZDCHF.FXCM",
#     "NZDJPY.FXCM",
#     "NZDUSD.FXCM",
#     "USDCAD.FXCM",
#     "USDCHF.FXCM",
#     "USDJPY.FXCM",
# ]
# list_tickers = pd.read_clipboard()["Ticker"].tolist()

# list_tickers = set(list_tickers)
list_tickers = [l.decode("utf-8") for l in list_tickers]

In [12]:
hist_conn = iq.HistoryConn(name="hist-conn")
hist_listener = iq.VerboseIQFeedListener("History Bar Listener")
hist_conn.add_listener(hist_listener)

dict_result = {}

with iq.ConnConnector([hist_conn]) as connector:
    for ticker in list_tickers:
        try:
            print(f"Processing {ticker}")
            d_start_date = pd.to_datetime(start_date).date()
            d_end_date = pd.to_datetime(end_date).date()
            daily_data = hist_conn.request_daily_data_for_dates(ticker, d_start_date, d_end_date, ascend=True)
            df_daily = pd.DataFrame(daily_data).rename(columns=DICT_NAME_MAPPER).set_index(DATE_NAME)
            if ".FXCM" in ticker:
                ticker = ticker.replace(".FXCM", "")
            dict_result[ticker] = df_daily
        except (iq.NoDataError, iq.UnauthorizedError) as err:
            print(f"Wrong Data {ticker}")

Processing KITT
Processing DC
Processing EFAS
Processing HAUS
Processing GGZ
Processing STE
Processing CBRL
Processing SHO
Processing IVDA
Processing IWX
Processing OCC
Processing SPCZ
Processing GGLS
Processing BKD
Processing CPIX
Processing GENY
Processing HE
Processing ELEV
Processing BLW
Processing LQDI
Processing IRWD
Processing FMIL
Processing PHT
Processing IWM
Processing RWK
Processing TRVN
Processing NVTA
Processing SPAX
Processing BFIT
Processing HTOO
Processing CHII
Processing WLKP
Processing DFAI
Processing GTH
Processing LAW
Processing SBIO
Processing SMLF
Processing ILDR
Processing PZZA
Processing AXR
Processing DJCO
Processing ECVT
Processing VNCE
Processing DPCS
Processing KBH
Processing ARHS
Processing TSQ
Processing GRC
Processing LBTYA
Processing CCRD
Processing RDVI
Processing ARLP
Processing ZBRA
Processing HGTY
Processing DK
Processing NHWK
Processing CHY
Processing PALI
Processing JVA
Processing LEG
Processing KOSS
Processing DRIV
Processing MBTC
Processing TY
Pr

RuntimeError: Request: HDT,test,20230913,20231122,,1,H_0000000981,100
, Error: Invalid symbol format.

In [ ]:
list_dfs_low = []
list_dfs_high = []
for ticker in dict_result.keys():
    list_dfs_high.append(dict_result[ticker][HIGH_NAME].to_frame(ticker))
    list_dfs_low.append(dict_result[ticker][LOW_NAME].to_frame(ticker))
df_high = pd.concat(list_dfs_high, axis=1)
df_low = pd.concat(list_dfs_low, axis=1)

In [ ]:
s = (df_high - df_low).tail(30).mean() + (df_high - df_low).tail(30).std()

In [ ]:
for ticker in s.index:
    redis_server[ticker] = s[ticker].round(5)